In [80]:
import requests
import pandas as pd
import json

In [71]:
# API endpoint
url = "https://api.elsevier.com/content/search/scopus/"
# AND SUBJAREA(AGRI)

In [72]:
field_table = pd.read_csv('./Abbreviation.csv')

In [73]:
field_list = field_table['ชื่อย่อ'].to_list()

In [74]:
eid_field_list = []
sum = 0

In [75]:
for i in field_list:
    #First Check For Amount
    params = {
        "query": "AFFILORG(Chulalongkorn University) AND PUBYEAR = 2017 AND SUBJAREA ( +" + i.lower() + " )",
        "field": "eid",
        "start": 0
    }
    headers = {
        "Accept": "application/json",
        "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
    }

    document_count = 0

    response = requests.get(url, headers=headers, params=params)

    if(response.status_code == 200):
        document_count = int(response.json()['search-results']['opensearch:totalResults'])

    sum += document_count

    start_index = 0
    while (start_index < document_count):
        params = {
        "query": "AFFILORG(Chulalongkorn University) AND PUBYEAR = 2017 AND SUBJAREA ( +" + i.lower() + " )",
        "field": "eid",
        "start": start_index
        }

        headers = {
            "Accept": "application/json",
            "X-ELS-APIKey": "fdfa191b9599a981a891f1f6d55bdc95",
        }

        response = requests.get(url, headers=headers, params=params)
        
        if(response.status_code == 200):
            res_json = response.json()
            documents = res_json['search-results']['entry']
            for document in documents:
                print(document['eid'], i)
                eid_field_list.append([document['eid'], i])
    
        start_index += 25
                




2-s2.0-85038429957 AGRI
2-s2.0-85029811763 AGRI
2-s2.0-85052976005 AGRI
2-s2.0-85047899907 AGRI
2-s2.0-85044253500 AGRI
2-s2.0-85040114563 AGRI
2-s2.0-85032829181 AGRI
2-s2.0-85022066491 AGRI
2-s2.0-85036668130 AGRI
2-s2.0-85053773163 AGRI
2-s2.0-85037706907 AGRI
2-s2.0-85037705620 AGRI
2-s2.0-85037691507 AGRI
2-s2.0-85037691271 AGRI
2-s2.0-85037688660 AGRI
2-s2.0-85037677307 AGRI
2-s2.0-85034116329 AGRI
2-s2.0-85026467782 AGRI
2-s2.0-85038430003 AGRI
2-s2.0-85035243322 AGRI
2-s2.0-85035229201 AGRI
2-s2.0-85032919230 AGRI
2-s2.0-85032026998 AGRI
2-s2.0-85024373885 AGRI
2-s2.0-85014239337 AGRI
2-s2.0-85020897462 AGRI
2-s2.0-85032581552 AGRI
2-s2.0-85032342303 AGRI
2-s2.0-85031911705 AGRI
2-s2.0-85031809054 AGRI
2-s2.0-85031784906 AGRI
2-s2.0-85031711640 AGRI
2-s2.0-85029788720 AGRI
2-s2.0-85028771275 AGRI
2-s2.0-85028541903 AGRI
2-s2.0-85028327212 AGRI
2-s2.0-85028023806 AGRI
2-s2.0-85020920795 AGRI
2-s2.0-85018586734 AGRI
2-s2.0-85010689164 AGRI
2-s2.0-85072768814 AGRI
2-s2.0-850204603

In [91]:
eid_field_list_df = pd.DataFrame(eid_field_list, columns=['Eid', 'Field'])

,Eid,Field
0,2-s2.0-85038429957,AGRI
1,2-s2.0-85029811763,AGRI
2,2-s2.0-85052976005,AGRI
3,2-s2.0-85047899907,AGRI
4,2-s2.0-85044253500,AGRI
...,...,...
4020,2-s2.0-85015335911,VETE
4021,2-s2.0-85014089683,VETE
4022,2-s2.0-85013374519,VETE
4023,2-s2.0-84995751226,VETE


In [81]:
with open('./PAPAER_SCRAPING.json') as d:
    data = json.load(d)

In [89]:
scrape_eid_list = []

for i in data:
    scrape_eid_list.append(i['data']['Eid'])

In [132]:
eid_field_list_df = pd.DataFrame(eid_field_list, columns=['Eid', 'Field_Abbv'])

In [133]:
eid_field_list_df = eid_field_list_df[eid_field_list_df['Eid'].isin(scrape_eid_list)]

In [134]:
field_table = field_table.rename(columns={'ชื่อย่อ' : "Field_Abbv", "ชื่อเต็ม" : "Subject_Field"})

In [135]:
eid_field_list_df = eid_field_list_df.merge(field_table, how='inner', on='Field_Abbv')

In [137]:
eid_field_list_df = eid_field_list_df.drop(columns='Field_Abbv')

In [139]:
eid_field_list_df.to_csv('scrabe_subject_field.csv')